<a href="https://colab.research.google.com/github/DonRoboto/MachineLearning/blob/master/NaiveBayes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detector de Spam

In [3]:
# Importar la libreria Pandas 
import pandas as pd

# Dataset de https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table('sample_data/SMSSpamCollection', 
                   sep='\t', 
                   names=['label','sms_message'])

# Visualización de las 5 primeras filas
df.head(10)

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


convertiremos las etiquetas en variables binarias, 0 representará ‘ham’ y 1 representará ‘spam’.

In [10]:
# Conversion
df['label'] = df.label.map({'ham':0, 'spam':1})

# Visualizar las dimensiones de los datos
df.shape

(5572, 2)